In [1]:
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='ticks', context='talk')

In [2]:
path = r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\probability maps\\'
allFiles=glob.glob(path+"*.csv")
frame=pd.DataFrame
listed=[]
for file in allFiles:
    df = pd.read_csv(file, index_col=[0])
    listed.append(df)
frame = pd.concat(listed)
crustal_abundance=pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\crustal_abundance.csv',encoding = "ISO-8859-1")

In [18]:
huc12=gpd.read_file(r'J:\Geology\WSGS\GIS\huc_12.shp') #reads in the hydraulic basin units and adds an area column
huc12= huc12.to_crs({'init': 'epsg:3732'})


counties = gpd.read_file(r'J:\Geology\WSGS\GIS\County_boundaries_polys.shp')
counties = counties.to_crs({'init': 'epsg:3732'})

roads = gpd.read_file(r'J:\Geology\WSGS\GIS\Highway_road.shp')
roads = roads.to_crs({'init': 'epsg:3732'})


In [4]:
frame['HUC_12']=frame.HUC_12.astype(str)

In [5]:
CaptainHucstable = huc12.merge(frame, on='HUC_12' )
pruned = CaptainHucstable[CaptainHucstable['prob']>0]

In [8]:
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interact
%matplotlib nbagg

formations = sorted(CaptainHucstable['formation'].unique().tolist())
elements = sorted(CaptainHucstable['element'].unique().tolist())
 
def view(formation='', element='', concentration=''):
    
    try:
        fig, ax = plt.subplots()
        pruned[(pruned['formation']==formation) & (pruned['element']==element) &
               (pruned['bins_ppm']>concentration)].plot(ax=ax, 
                    column='prob', cmap='OrRd', legend=True)
        huc12.plot(ax=ax, alpha=0.1)
    except:
        print('This combination does not exist')
    


w = widgets.Select(options=formations)
v = widgets.Select(options=elements)
u = widgets.FloatText(value=1)
interact(view, formation=w, element=v, concentration=u)
#interact(view, y=v)

interactive(children=(Select(description='formation', options=('Absaroka Volcanic Supergroup: Hominy Peak Form…

<function __main__.view>

In [46]:
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interact
%matplotlib nbagg

formations = sorted(CaptainHucstable['formation'].unique().tolist())
elements = sorted(CaptainHucstable['element'].unique().tolist())
 
def view2(formation='', element='', probability=''):
    
    try:
        fig, ax = plt.subplots()
        counties.plot(ax=ax, alpha=0.8, color='white', edgecolor='black')
        #roads.plot(ax=ax, color='black')

        pruned[(pruned['formation']==formation) & (pruned['element']==element) &
               (pruned['bins_ppm']>=crustal_abundance[element][5]*5) & (
               pruned['prob']> probability)].plot(ax=ax, 
                    column='prob', cmap='OrRd', legend=True)
    except:
        print('This combination does not exist')
    


w = widgets.Select(options=formations)
v = widgets.Select(options=elements)
u = widgets.FloatSlider(min=0.1,max=1.0,step=0.1,value=0.5)


interact(view2, formation=w, element=v, probability=u)
#interact(view, y=v)

interactive(children=(Select(description='formation', options=('Absaroka Volcanic Supergroup: Hominy Peak Form…

<function __main__.view2>